In [ ]:
import pandas as pd

test_predictions_df = pd.read_csv("test_predictions.csv")

columns_to_drop = ['embedding', 'detected_emotion', 'subject', 'verb', 'object', 'emotion_type']
test_predictions_df = test_predictions_df.drop(columns=columns_to_drop, errors='ignore')

print(test_predictions_df.head())



            id                                          clause speaker  \
0  tr_9708_1_1                      The blake's got divorced .       A   
1  tr_9708_2_1                                        Really ?       B   
2  tr_9708_2_2                                           Why ?       B   
3  tr_9708_3_1                                            Mr .       A   
4  tr_9708_3_2  black has been getting a little around aside .       A   

   predicted_label predicted_class  true_label true_class  
0                2         neither           2    neither  
1                0         emotion           2    neither  
2                0         emotion           2    neither  
3                3            both           2    neither  
4                0         emotion           2    neither  


In [9]:
test_predictions_df.to_csv("test_predictions.csv", index=False)

In [ ]:
import pandas as pd

test_predictions_df = pd.read_csv("test_predictions.csv")
pairs = []
previous_clause = None
previous_conv_id = None

for i, row in test_predictions_df.iterrows():
    clause_id = row['id']
    clause = row['clause']
    speaker = row['speaker']
    predicted_class = row['true_class']
    conv_id = clause_id.split('_')[0]  

    if previous_conv_id != conv_id:
        previous_clause = None
        previous_conv_id = conv_id

    if predicted_class == 'emotion' and previous_clause is not None and previous_conv_id == conv_id:
        pairs.append({
            'id': previous_clause['id'] + " --> " + clause_id,
            'emotion': clause,  
            'cause': previous_clause['clause'],  
            'speaker': previous_clause['speaker'] + " --> " + speaker
        })
        
        previous_clause = None
    elif predicted_class == 'both':
        if previous_clause is not None:
            pairs.append({
                'id': previous_clause['id'] + " --> " + clause_id,
                'emotion': clause,  
                'cause': previous_clause['clause'],  
                'speaker': previous_clause['speaker'] + " --> " + speaker
            })
        previous_clause = row
        previous_conv_id = conv_id
    elif predicted_class == 'cause':
        previous_clause = row
        previous_conv_id = conv_id

pairs_df = pd.DataFrame(pairs)

pairs_df.to_csv("emotion_cause_pairs.csv", index=False)
print(pairs_df.head())




                            id  \
0  tr_9708_5_4 --> tr_9708_6_1   
1  tr_9708_6_1 --> tr_9708_6_2   
2  tr_9708_8_3 --> tr_9708_9_1   
3    te_935_1_1 --> te_935_2_1   
4    te_935_2_1 --> te_935_2_2   

                                             emotion  \
0                      Well , I'm really surprised .   
1  You are not doing anything behind my back , ar...   
2                         You are not funny either .   
3                                             Cool !   
4                                 Flashy red lines !   

                                               cause  speaker  
0  Starting almost right after they married 20 ye...  A --> B  
1                      Well , I'm really surprised .  B --> B  
2  I guess that means - - except me to tell my lo...  B --> A  
3                  I'm so glad I bought this watch !  A --> B  
4                                             Cool !  B --> B  
